
# Chillstay Chatbot — Firebase Edition (Notebook)

Notebook này là phiên bản chuyển từ file script bạn cung cấp sang **Jupyter Notebook** để dễ chỉnh sửa và chạy từng phần.
Tôi đã giữ nguyên logic chính, thêm một số *compatibility checks*, và đề xuất/fix các chỗ có thể gây lỗi khi chạy (xem ô "Ghi chú & Fixes").

**Lưu ý:** không thể kết nối Firebase hoặc OpenAI trong môi trường này; notebook sẽ tạo file `.ipynb` cho bạn tải xuống và chạy trên máy có cấu hình (credentials) phù hợp.


In [2]:

# PHẦN 1: IMPORT VÀ SETUP
import os
import getpass
import json
from typing import Type, Optional, List, Dict, Any

# Firebase
import firebase_admin
from firebase_admin import credentials, firestore

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import ChatPromptTemplate, FewShotPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.tools import BaseTool

from pydantic import BaseModel, Field
import gradio as gr


In [3]:
# Thiết lập Google API Key
if 'OPENAI_API_KEY' not in os.environ:
  os.environ['GOOGLE_API_KEY'] = getpass.getpass('Google API Key: ')

Google API Key:  ········


In [ ]:
# PHẦN 2: KHỞI TẠO FIREBASE
def initialize_firebase(service_account_path):
    try:
        if not firebase_admin._apps:
            cred = credentials.Certificate(service_account_path)
            firebase_admin.initialize_app(cred)
        db = firestore.client()
        print("✓ Firebase đã được khởi tạo thành công!")
        return db
    except Exception as e:
        print(f"✗ Lỗi khởi tạo Firebase: {e}")
        return None

db = initialize_firebase("/path/to/your/firebase-key.json")


In [ ]:

# PHẦN 3 & 4: TOOLS - Query và Aggregation (có hỗ trợ toán tử cho filters)
class FirebaseQueryInput(BaseModel):
    collection: str = Field(..., description="Tên collection cần query (hotels, rooms, bookings, users, etc.)")
    filters: Dict[str, Any] = Field(default={}, description="Điều kiện lọc dạng dict {field: value} hoặc {field: {operator: value}}")
    limit: int = Field(default=10, description="Số lượng kết quả tối đa")
    order_by: Optional[str] = Field(None, description="Trường dùng để sắp xếp")

class FirebaseQueryTool(BaseTool):
    name: str = "firebase_query"
    description: str = "Truy vấn Firestore cho Chillstay"
    args_schema: Type[BaseModel] = FirebaseQueryInput
    return_direct: bool = False

    def __init__(self, db, **kwargs):
        super().__init__(**kwargs)
        self._db = db

    def _apply_filter(self, query, field, value):
        # Hỗ trợ value là dict: { "<": 100 } hoặc giá trị trực tiếp => '='
        if isinstance(value, dict):
            # Lấy operator và operand từ dict
            if len(value) != 1:
                raise ValueError("Filter dict phải có đúng 1 cặp operator:value")
            op, val = next(iter(value.items()))
            op_map = {"<": "<", "<=": "<=", ">": ">", ">=": ">=", "!=": "!=", "==": "=="}
            if op not in op_map:
                raise ValueError(f"Operator không hỗ trợ: {op}")
            query = query.where(field, op_map[op], val)
        else:
            query = query.where(field, "==", value)
        return query

    def _run(self, collection: str, filters: Dict[str, Any] = {}, limit: int = 10, order_by: Optional[str] = None, run_manager: Optional[Any] = None) -> str:
        try:
            query = self._db.collection(collection)

            for field, value in filters.items():
                query = self._apply_filter(query, field, value)

            if order_by:
                try:
                    query = query.order_by(order_by, direction=firestore.Query.DESCENDING)
                except Exception:
                    # fallback đơn giản
                    query = query.order_by(order_by)

            query = query.limit(limit)
            docs = query.stream()

            results = []
            for doc in docs:
                data = doc.to_dict()
                data['id'] = doc.id
                results.append(data)

            if not results:
                return f"Không tìm thấy dữ liệu trong collection '{collection}' với điều kiện: {filters}"

            return json.dumps(results, ensure_ascii=False, indent=2)
        except Exception as e:
            return f"Lỗi khi query Firebase: {str(e)}"


class FirebaseAggregationInput(BaseModel):
    collection: str = Field(...)
    operation: str = Field(..., description="count, avg, min, max, sum")
    field: Optional[str] = Field(None)
    filters: Dict[str, Any] = Field(default={})

class FirebaseAggregationTool(BaseTool):
    name: str = "firebase_aggregate"
    description: str = "Thống kê trên Firestore"
    args_schema: Type[BaseModel] = FirebaseAggregationInput
    return_direct: bool = False

    def __init__(self, db, **kwargs):
        super().__init__(**kwargs)
        self._db = db

    def _run(self, collection: str, operation: str, field: Optional[str] = None, filters: Dict[str, Any] = {}, run_manager: Optional[Any] = None) -> str:
        try:
            query = self._db.collection(collection)
            for key, value in filters.items():
                if isinstance(value, dict):
                    # re-use same operator handling as query tool
                    if len(value) != 1:
                        raise ValueError("Filter dict phải có đúng 1 cặp operator:value")
                    op, val = next(iter(value.items()))
                    query = query.where(key, op, val)
                else:
                    query = query.where(key, "==", value)

            docs = list(query.stream())

            if operation == 'count':
                return f"Số lượng: {len(docs)}"

            if not field:
                return "Lỗi: Phải chỉ định field cho phép toán này"

            values = []
            for doc in docs:
                data = doc.to_dict()
                if field in data and data[field] is not None:
                    try:
                        values.append(float(data[field]))
                    except Exception:
                        # bỏ qua giá trị không chuyển được
                        pass

            if not values:
                return f"Không tìm thấy giá trị cho field '{field}'"

            if operation == 'avg':
                result = sum(values) / len(values)
                return f"Trung bình {field}: {result:.2f}"
            elif operation == 'min':
                return f"Nhỏ nhất {field}: {min(values)}"
            elif operation == 'max':
                return f"Lớn nhất {field}: {max(values)}"
            elif operation == 'sum':
                return f"Tổng {field}: {sum(values)}"
            else:
                return f"Phép toán '{operation}' không được hỗ trợ"
        except Exception as e:
            return f"Lỗi: {str(e)}"


In [ ]:

# PHẦN 5 & 6: SEMANTIC SEARCH + TẠO VECTOR STORE
class SemanticSearchInput(BaseModel):
    query: str = Field(..., description="Câu hỏi tìm kiếm ngữ nghĩa")

class SemanticSearchTool(BaseTool):
    name: str = "semantic_search"
    description: str = "Tìm kiếm ngữ nghĩa trên tên, mô tả, tiện nghi"
    args_schema: Type[BaseModel] = SemanticSearchInput
    return_direct: bool = False

    def __init__(self, retriever, **kwargs):
        super().__init__(**kwargs)
        self._retriever = retriever

    def _run(self, query: str, run_manager: Optional[Any] = None) -> str:
        try:
            results = self._retriever.get_relevant_documents(query)
        except Exception:
            results = self._retriever.retrieve(query)
        if not results:
            return "Không tìm thấy kết quả phù hợp"
        output = []
        for doc in results[:5]:
            # doc.page_content hoặc doc.content tuỳ object
            content = getattr(doc, 'page_content', None) or getattr(doc, 'content', str(doc))
            output.append(content)
        return "\n\n---\n\n".join(output)


def create_hotel_vector_store(db, embeddings):
    \"\"\"Tạo vectorstore từ hotels collection\"\"\"
    print("Đang tạo vector store từ Firebase...")
    hotels_ref = db.collection('hotels')
    hotels = hotels_ref.stream()

    texts = []
    metadatas = []

    for hotel in hotels:
        data = hotel.to_dict()
        text = f\"\"\"\nTên: {data.get('name', '')}\nThành phố: {data.get('city', '')}\nQuốc gia: {data.get('country', '')}\nRating: {data.get('rating', 0)}\nSố đánh giá: {data.get('numberOfReviews', 0)}\n\"\"\"
        texts.append(text)
        metadatas.append({'id': hotel.id, 'name': data.get('name', ''), 'city': data.get('city', ''), 'country': data.get('country', '')})

    if not texts:
        print("⚠ Không có dữ liệu hotel trong Firebase")
        return None

    # Tùy version, Chroma.from_texts signature khác nhau (embedding_function vs embedding)
    try:
        vectorstore = Chroma.from_texts(texts=texts, metadatas=metadatas, embedding=embeddings)
    except Exception as e:
        try:
            vectorstore = Chroma.from_texts(texts, metadatas=metadatas, embedding_function=embeddings)
        except Exception as e2:
            raise RuntimeError(f"Không thể tạo Chroma vectorstore: {e} | {e2}")

    print(f"✓ Đã tạo vector store với {len(texts)} hotels")
    return vectorstore


In [ ]:

# PHẦN 7 & 8: FEW-SHOT EXAMPLES + TẠO AGENT (cố gắng tương thích với nhiều version)
examples = [
    {"input": "Có bao nhiêu khách sạn ở Hà Nội?", "query": "Sử dụng firebase_aggregate: - collection: hotels - operation: count - filters: {\"city\": \"Hà Nội\"}"},
    {"input": "Tìm 5 khách sạn có rating cao nhất", "query": "Sử dụng firebase_query: - collection: hotels - order_by: rating - limit: 5"},
    {"input": "Phòng nào có giá dưới 500k?", "query": "Lọc trong firebase_query: - collection: rooms - filters: {\"price\": {\"<\": 500000}} - limit: 10"},
    {"input": "Rating trung bình của các khách sạn ở TP.HCM?", "query": "Sử dụng firebase_aggregate: - collection: hotels - operation: avg - field: rating - filters: {\"city\": \"TP.HCM\"}"},
]

def create_chillstay_agent(db):
    llm = None
    if ChatOpenAI is not None:
        try:
            llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
        except Exception:
            try:
                llm = ChatOpenAI()  # fallback to default
            except Exception:
                llm = None

    embeddings = None
    if OpenAIEmbeddings is not None:
        try:
            embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
        except Exception:
            try:
                embeddings = OpenAIEmbeddings()
            except Exception:
                embeddings = None

    firebase_query_tool = FirebaseQueryTool(db=db)
    firebase_agg_tool = FirebaseAggregationTool(db=db)

    vectorstore = None
    if embeddings is not None and Chroma is not None:
        try:
            vectorstore = create_hotel_vector_store(db, embeddings)
        except Exception as e:
            print("Không thể tạo vectorstore:", e)
            vectorstore = None

    tools = [firebase_query_tool, firebase_agg_tool]

    if vectorstore:
        retriever = vectorstore.as_retriever(search_kwargs={'k': 5})
        semantic_tool = SemanticSearchTool(retriever=retriever)
        tools.append(semantic_tool)

    # NOTE: phần tạo prompt/agent phụ thuộc rất lớn vào version của langchain.
    # Ở đây ta cố gắng tạo agent bằng API-level đơn giản, nhưng nếu lỗi, bạn cần cập nhật theo version langchain hiện hành.
    try:
        from langchain.agents import create_openai_tools_agent, AgentExecutor
        agent = create_openai_tools_agent(llm, tools, None)
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=10)
    except Exception:
        # Fallback nhẹ: đơn giản bọc tools thành object có run() trả chuỗi
        class SimpleAgentExecutor:
            def __init__(self, tools):
                self.tools = tools
            def run(self, input_str):
                return "Agent không được khởi tạo do khác version langchain. Hãy kiểm tra imports và phiên bản langchain."
        agent_executor = SimpleAgentExecutor(tools)

    return agent_executor


In [ ]:

# PHẦN 9: HÀM CHAT (cố gắng tương thích với nhiều interface của agent_executor)
def chat(user_message, history, agent_executor):
    if history is None:
        history = []
    try:
        # Thử nhiều cách gọi khác nhau tuỳ version langchain
        result = None
        if hasattr(agent_executor, "invoke"):
            try:
                result = agent_executor.invoke({"input": user_message})
            except Exception:
                result = agent_executor.invoke(user_message)
        elif hasattr(agent_executor, "run"):
            result = agent_executor.run(user_message)
        elif callable(agent_executor):
            result = agent_executor({"input": user_message})
        else:
            result = "Agent không hỗ trợ phương thức gọi tự động. Kiểm tra AgentExecutor."

        # Lấy bot_reply an toàn (result có thể là dict hoặc str)
        bot_reply = None
        if isinstance(result, dict):
            bot_reply = result.get('output') or result.get('result') or str(result)
        elif isinstance(result, str):
            bot_reply = result
        else:
            try:
                bot_reply = str(result)
            except Exception:
                bot_reply = "Không thể lấy kết quả từ agent."

        history.append({ "role": "user", "content": user_message })
        history.append({ "role": "assistant", "content": bot_reply })

        return "", history
    except Exception as e:
        error_msg = f"Lỗi: {str(e)}"
        history.append({ "role": "user", "content": user_message })
        history.append({ "role": "assistant", "content": error_msg })
        return "", history


# PHẦN 10: MAIN - Khởi chạy Gradio (khi đã có db và agent_executor)
def launch_ui(agent_executor):
    with gr.Blocks() as demo:
        gr.Markdown("# 🏨 Chillstay Chatbot - Firebase Edition (Notebook)")
        gr.Markdown("Trợ lý AI giúp bạn tìm kiếm và quản lý khách sạn")
        chatbot = gr.Chatbot(type='messages', height=500)
        with gr.Row():
            txt = gr.Textbox(show_label=False, placeholder="Hỏi về khách sạn, phòng, đặt phòng...", scale=9)
            btn = gr.Button("Gửi", scale=1)
        txt.submit(lambda msg, hist: chat(msg, hist, agent_executor), [txt, chatbot], [txt, chatbot])
        btn.click(lambda msg, hist: chat(msg, hist, agent_executor), [txt, chatbot], [txt, chatbot])
        gr.Examples(examples=["Có bao nhiêu khách sạn?", "Tìm 5 khách sạn có rating cao nhất", "Phòng nào giá dưới 500k?", "Rating trung bình của khách sạn ở Hà Nội?" ], inputs=txt)
    demo.launch(share=True)

# Ví dụ (khi chạy locally)
# service_account_path = "/path/to/your/firebase-key.json"
# db = initialize_firebase(service_account_path)
# agent_executor = create_chillstay_agent(db)
# launch_ui(agent_executor)



## Ghi chú & Fixes (Những điểm cần chú ý khi chạy trên máy thực tế)

1. **Các import của LangChain**: LangChain thay đổi nhiều giữa các version — các module như `langchain_openai`, `langchain_core`, `langchain_community` có thể **không tồn tại**. Nếu bạn gặp lỗi import:
   - Kiểm tra version `pip show langchain` và đọc changelog hoặc docs tương ứng.
   - Thay thế `from langchain_openai import ChatOpenAI` bằng `from langchain.chat_models import ChatOpenAI` nếu cần.
   - Tương tự với `Chroma` và `OpenAIEmbeddings` (chroma có thể nằm trong `langchain.vectorstores` hoặc `langchain_community.vectorstores`).

2. **Chroma.from_texts signature**: Tùy version, tên tham số embedding có thể là `embedding` hoặc `embedding_function`. Notebook đã thử fallback, nhưng nếu vẫn lỗi, kiểm tra docs của phiên bản Chroma bạn đang dùng.

3. **Agent API**: Hàm tạo agent (`create_openai_tools_agent`, `AgentExecutor`, `invoke`, `run`) thay đổi giữa các release. Nếu agent không khởi tạo, tốt nhất là xem tài liệu langchain version bạn cài và thay thế phần tạo agent bằng một `Tool`-based agent hoặc custom loop gọi tools.

4. **Filters operator**: Tôi đã bổ sung hỗ trợ cho filters dạng `{"price": {"<": 500000}}`. Hãy đảm bảo dữ liệu `price` trong Firestore là numeric để so sánh.

5. **Quyền truy cập và credentials**:
   - Firebase: cung cấp path tới file service account JSON khi gọi `initialize_firebase()`.
   - OpenAI: export `OPENAI_API_KEY` trước khi chạy notebook, hoặc nhập bằng input (getpass).

6. **Gradio**: Phiên bản gradio đôi khi thay đổi API (ví dụ Chatbot signature). Nếu lỗi, kiểm tra version gradio và thay đổi tham số tương ứng.

7. **Kiểm tra bước thử nghiệm**: Trong script gốc có phần test agent bằng `agent_executor.invoke`. Tôi đã chuẩn hóa thành thử nhiều cách gọi để tránh lỗi phương thức không tồn tại.

Nếu bạn muốn, tôi có thể tiếp tục **chạy static checks** hoặc **sửa đổi code theo exact langchain version** bạn đang sử dụng — nhưng để làm việc đó tôi sẽ cần bạn cho biết version (ví dụ `pip show langchain`, `pip show chromadb`) hoặc cho phép tôi xem output lỗi khi bạn chạy trên máy.
